In [ ]:
import colamatch as clm
from colamatch import constellation_matching as cm
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
%matplotlib notebook

## Define filenames and read images and landmarks

In [ ]:
s_fixed = 1
s_moving = 2
fixed_file = "../data/section_%s.tif"%(str(s_fixed).zfill(4))
moving_file = "../data/section_%s.tif"%(str(s_moving).zfill(4)) 
fixed_landmarks = "../data/section_%s.landmarks.txt"%(str(s_fixed).zfill(4))
moving_landmarks = "../data/section_%s.landmarks.txt"%(str(s_moving).zfill(4))

In [ ]:
# read landmarks and images
l_fixed = np.loadtxt(fixed_landmarks).astype('int')
l_moving = np.loadtxt(moving_landmarks).astype('int')
img_fixed = cv2.imread(fixed_file,0)
img_moving = cv2.imread(moving_file,0)
print(len(l_fixed), len(l_moving))

## Perform constellation matching

In [ ]:
# working settings: num_samples=1500, radius=0.13, lamda=2, ransac=0.1
# working settings: num_samples=10000, radius=0.1, lamda=2, ransac=0.1 # oder radius=0.8
# warum so schlechte matches fuer grosses num_matches? radius muss kleiner werden fuer grosse num_samples...
num_samples = 10000 #100000
runtimes = {}
#for num_samples in range(1000,100001,5000):
c1_fixed = clm.ExhaustiveSampler(len(l_fixed),4)
c1_moving = clm.ExhaustiveSampler(len(l_moving),4)
c2_fixed = clm.RandomSampler(len(l_fixed), 4, num_samples)
c2_moving = clm.RandomSampler(len(l_moving), 4, num_samples)
start = time.time()
matches = cm.match(l_fixed, l_moving, c2_fixed, c2_moving, radius=0.08, lamda=2, ransac=0.1)
print("runtime for num_samples=%s: %f" % (num_samples,time.time()-start))
runtimes[num_samples] = time.time()-start

In [ ]:
plt.figure()
plt.plot(list(runtimes.keys()), list(runtimes.values()))
plt.xlabel("num_samples")
plt.ylabel("seconds")
plt.plot()
#plt.savefig('random_runtimes.png')

In [ ]:
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(1,2)
axs[0].imshow(img_fixed, cmap='gray')
axs[1].imshow(img_moving, cmap='gray')
axs[0].plot(l_fixed[:,0], l_fixed[:,1],'w+',ms=15)
axs[1].plot(l_moving[:,0], l_moving[:,1],'w+',ms=15)
axs[0].axis('off')
axs[1].axis('off')
for i,candidate in enumerate(matches): 
    axs[0].plot(candidate[0,0], candidate[0,1], c=cmap(i), marker='o')
    axs[1].plot(candidate[1,0], candidate[1,1], c=cmap(i), marker='o')
fig.show()

## Match real vessel detections

In [ ]:
# read detections from 1529_vessels.h5 and 1530_vessels.h5
import h5py
l_fixed = h5py.File("../data/1529_vessels.h5")["data"][:,:2]
l_moving = h5py.File("../data/1530_vessels.h5")["data"][:,:2]
print(l_fixed.shape[0], l_moving.shape[0])

In [ ]:
num_samples = 100000
#c1_fixed = clm.ExhaustiveSampler(len(l_fixed),4)  # MemoryError
#c1_moving = clm.ExhaustiveSampler(len(l_moving),4)  # MemoryError
c2_fixed = clm.RandomSampler(len(l_fixed), 4, num_samples)
c2_moving = clm.RandomSampler(len(l_moving), 4, num_samples)
start = time.time()
matches = cm.match(l_fixed, l_moving, c2_fixed, c2_moving, radius=0.035, lamda=2, ransac=0.1)
print("runtime for num_samples=%s: %f" % (num_samples,time.time()-start))

In [ ]:
roifile = h5py.File("../data/B20_V1_l_rois.h5")
roi_fixed = roifile["1529/roi"][:]
roi_moving = roifile["1530/roi"][:]
print(roi_fixed, roi_moving)
scale = 0.1

In [ ]:
img_fixed = cv2.imread("../data/B20_V1_l_Kai_1529_scale0.1.tif",0)
img_moving = cv2.imread("../data/B20_V1_l_Kai_1530_scale0.1.tif",0)
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(2,1)
axs[0].imshow(img_fixed, cmap='gray')
axs[1].imshow(img_moving, cmap='gray')
# plot landmarks - roi-offset * scale according to downscaled images (0.1)
axs[0].plot((l_fixed[:,0]-roi_fixed[0,0])*scale, (l_fixed[:,1]-roi_fixed[0,1])*scale,'w+',ms=5)
axs[1].plot((l_moving[:,0]-roi_moving[0,0])*scale, (l_moving[:,1]-roi_moving[0,1])*scale,'w+',ms=5)
axs[0].axis('off')
axs[1].axis('off')
for i,match in enumerate(matches): 
    # plot matched coordinates - roi-offset * scale according to downscaled images (0.1)
    axs[0].plot((match[0,0]-roi_fixed[0,0])*scale, (match[0,1]-roi_fixed[0,1])*scale, c=cmap(i), marker='o')
    axs[1].plot((match[1,0]-roi_moving[0,0])*scale, (match[1,1]-roi_moving[0,1])*scale, c=cmap(i), marker='o')
fig.show()